# Loading data

In [1]:
import pandas as pd

df_new = pd.read_csv('df_new.csv', index_col='file_name')
df_new.head()

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127,Sarcasm
file_name,,,,,,,,,,,,,,,,,,,,,
1_10004_u.wav,158.571426,3.714286,190.000000,121.428574,193.142853,77.428574,113.142860,80.285713,117.571426,198.285721,...,208.285721,81.142860,150.428574,152.428574,192.857147,229.857147,60.285713,110.571426,255.0,0.0
1_10004_u.wav_add_noise.wav,156.000000,5.714286,187.428574,97.428574,207.571426,86.857140,109.142860,83.857140,122.142860,215.857147,...,236.428574,133.428574,110.714287,113.857140,186.000000,189.571426,49.428570,85.571426,255.0,0.0
1_10004_u.wav_pitch_shift.wav,161.714279,11.571428,189.000000,127.142860,204.142853,84.714287,100.571426,98.000000,125.428574,204.000000,...,215.428574,107.142860,140.285721,91.142860,166.428574,196.000000,76.714287,85.142860,255.0,0.0
1_10004_u.wav_time_stretch.wav,162.500000,12.500000,196.500000,126.166664,204.333328,84.333336,113.000000,83.333336,118.666664,212.666672,...,192.666672,74.833336,181.666672,136.833328,126.833336,159.666672,50.000000,85.333336,255.0,0.0
1_10009_u.wav,169.000000,10.600000,219.199997,148.199997,194.800003,61.799999,173.000000,48.400002,128.000000,194.000000,...,236.000000,1.400000,248.800003,212.000000,177.000000,232.399994,60.000000,207.000000,255.0,0.0


# Neural Network

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Scale the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_new.drop(columns=['Sarcasm']))
y = df_new['Sarcasm'].astype(int)

# Split data into train
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42) 

# Define the model
model = Sequential()
model.add(Dense(128, input_dim=128, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))
model.add(Dropout(0.6))

model.add(Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))
model.add(Dropout(0.6))

model.add(Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))
model.add(Dropout(0.6))

model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Set callback functions to early stop training
callbacks = [EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)]

# Train the model
history = model.fit(X_train, y_train, epochs=500, batch_size=128, callbacks=callbacks, validation_split=0.2)

# Predict on the augmented test set
y_pred_prob = model.predict(X_test)
y_pred = np.round(y_pred_prob).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Epoch 1/500
25/25 [==============================] - 2s 10ms/step - loss: 1.0981 - accuracy: 0.5157 - val_loss: 0.7316 - val_accuracy: 0.5039
Epoch 2/500
25/25 [==============================] - 0s 4ms/step - loss: 0.9044 - accuracy: 0.4862 - val_loss: 0.7326 - val_accuracy: 0.4793
Epoch 3/500
25/25 [==============================] - 0s 4ms/step - loss: 0.8098 - accuracy: 0.5034 - val_loss: 0.7315 - val_accuracy: 0.4922
Epoch 4/500
25/25 [==============================] - 0s 4ms/step - loss: 0.7877 - accuracy: 0.5066 - val_loss: 0.7313 - val_accuracy: 0.4948
Epoch 5/500
25/25 [==============================] - 0s 4ms/step - loss: 0.7529 - accuracy: 0.5109 - val_loss: 0.7304 - val_accuracy: 0.4896
Epoch 6/500
25/25 [==============================] - 0s 3ms/step - loss: 0.7534 - accuracy: 0.5037 - val_loss: 0.7301 - val_accuracy: 0.5078
Epoch 7/500
25/25 [==============================] - 0s 3ms/step - loss: 0.7612 - accuracy: 0.4878 - val_loss: 0.7295 - val_accuracy: 0.5013
Epoch 8/500


Epoch 59/500
25/25 [==============================] - 0s 4ms/step - loss: 0.7160 - accuracy: 0.5391 - val_loss: 0.7225 - val_accuracy: 0.5091
Epoch 60/500
25/25 [==============================] - 0s 3ms/step - loss: 0.7170 - accuracy: 0.5468 - val_loss: 0.7222 - val_accuracy: 0.5104
Epoch 61/500
25/25 [==============================] - 0s 4ms/step - loss: 0.7187 - accuracy: 0.5400 - val_loss: 0.7222 - val_accuracy: 0.5091
Epoch 62/500
25/25 [==============================] - 0s 3ms/step - loss: 0.7133 - accuracy: 0.5433 - val_loss: 0.7219 - val_accuracy: 0.5104
Epoch 63/500
25/25 [==============================] - 0s 4ms/step - loss: 0.7163 - accuracy: 0.5468 - val_loss: 0.7213 - val_accuracy: 0.5130
Epoch 64/500
25/25 [==============================] - 0s 4ms/step - loss: 0.7154 - accuracy: 0.5446 - val_loss: 0.7212 - val_accuracy: 0.5104
Epoch 65/500
25/25 [==============================] - 0s 4ms/step - loss: 0.7132 - accuracy: 0.5553 - val_loss: 0.7216 - val_accuracy: 0.5065
Epoch 

25/25 [==============================] - 0s 3ms/step - loss: 0.6804 - accuracy: 0.6000 - val_loss: 0.7048 - val_accuracy: 0.5803
Epoch 117/500
25/25 [==============================] - 0s 3ms/step - loss: 0.6822 - accuracy: 0.5932 - val_loss: 0.7032 - val_accuracy: 0.5648
Epoch 118/500
25/25 [==============================] - 0s 3ms/step - loss: 0.6786 - accuracy: 0.6071 - val_loss: 0.7045 - val_accuracy: 0.5725
Epoch 119/500
25/25 [==============================] - 0s 4ms/step - loss: 0.6716 - accuracy: 0.6130 - val_loss: 0.7028 - val_accuracy: 0.5725
Epoch 120/500
25/25 [==============================] - 0s 4ms/step - loss: 0.6816 - accuracy: 0.6052 - val_loss: 0.7034 - val_accuracy: 0.5725
Epoch 121/500
25/25 [==============================] - 0s 4ms/step - loss: 0.6771 - accuracy: 0.6104 - val_loss: 0.7035 - val_accuracy: 0.5699
Epoch 122/500
25/25 [==============================] - 0s 3ms/step - loss: 0.6679 - accuracy: 0.6117 - val_loss: 0.7045 - val_accuracy: 0.5725
Epoch 123/500

25/25 [==============================] - 0s 3ms/step - loss: 0.6161 - accuracy: 0.6872 - val_loss: 0.6962 - val_accuracy: 0.5829
Epoch 174/500
25/25 [==============================] - 0s 3ms/step - loss: 0.6140 - accuracy: 0.6882 - val_loss: 0.6980 - val_accuracy: 0.5751
Epoch 175/500
25/25 [==============================] - 0s 3ms/step - loss: 0.6182 - accuracy: 0.6830 - val_loss: 0.7043 - val_accuracy: 0.5674
Epoch 176/500
25/25 [==============================] - 0s 3ms/step - loss: 0.6097 - accuracy: 0.6856 - val_loss: 0.7004 - val_accuracy: 0.5609
Epoch 177/500
25/25 [==============================] - 0s 4ms/step - loss: 0.6168 - accuracy: 0.6882 - val_loss: 0.6968 - val_accuracy: 0.5868
Epoch 178/500
25/25 [==============================] - 0s 3ms/step - loss: 0.6191 - accuracy: 0.6791 - val_loss: 0.7024 - val_accuracy: 0.5868
Epoch 179/500
25/25 [==============================] - 0s 3ms/step - loss: 0.6150 - accuracy: 0.6959 - val_loss: 0.7030 - val_accuracy: 0.5933
Epoch 180/500

#### Observation: Poor performance but better than traditional feature models

## Display Metric Function

In [3]:
def display_metrics(y_tes,y_pre):
    # Calculate metrics
    accuracy = accuracy_score(y_tes, y_pre)
    precision = precision_score(y_tes, y_pre)
    recall = recall_score(y_tes, y_pre)
    f1 = f1_score(y_tes, y_pre)

    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

# Classical Models

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier()
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
grid_rf.fit(X_train, y_train)

print("Best Parameters for Random Forest:", grid_rf.best_params_)

y_pred_rf = grid_rf.predict(X_test)

# Display metrics for XGBoost
print("RF Metrics:")
display_metrics(y_test, y_pred_rf)

Best Parameters for Random Forest: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
RF Metrics:
Accuracy: 0.6456876456876457
Precision: 0.6455696202531646
Recall: 0.6923076923076923
F1 Score: 0.668122270742358


In [7]:
import xgboost as xgb

xg = xgb.XGBClassifier()
param_grid_xg = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.8, 0.9, 1]
}

grid_xg = GridSearchCV(estimator=xg, param_grid=param_grid_xg, cv=5)
grid_xg.fit(X_train, y_train)

print("Best Parameters for XGBoost:", grid_xg.best_params_)

y_pred_xg = grid_xg.predict(X_test)

# Display metrics for XGBoost
print("XGBoost Metrics:")
display_metrics(y_test, y_pred_xg)


Best Parameters for XGBoost: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 500, 'subsample': 0.8}
XGBoost Metrics:
Accuracy: 0.6317016317016317
Precision: 0.6363636363636364
Recall: 0.665158371040724
F1 Score: 0.6504424778761061


In [8]:
from sklearn.svm import SVC

svm = SVC(probability=True)  # 'probability=True' to ensure you can retrieve probabilities
param_grid_svm = {
    'C': [0.1, 1, 10, 100], 
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'poly', 'sigmoid']
}

grid_svm = GridSearchCV(estimator=svm, param_grid=param_grid_svm, cv=5)
grid_svm.fit(X_train, y_train)

print("Best Parameters for SVM:", grid_svm.best_params_)

y_pred_svm = grid_svm.predict(X_test)

# Display metrics for SVM
print("SVM Metrics:")
display_metrics(y_test, y_pred_svm)


Best Parameters for SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
SVM Metrics:
Accuracy: 0.6876456876456877
Precision: 0.6899563318777293
Recall: 0.7149321266968326
F1 Score: 0.7022222222222222


### Observation: 
Out of the three models, XGBoost, RF and SVM, SVM is the best performing model. Let's check if the model is overfitting and tune accordingly

In [22]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Using the best parameters to train the SVM model
best_params_svm = {'C': 0.3, 'gamma': 0.01, 'kernel': 'rbf'}

svm_best = SVC(**best_params_svm)
svm_best.fit(X_train, y_train)

# Predict on the test set using the best model
y_pred_best_svm = svm_best.predict(X_test)

# Calculate metrics for the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
precision_best_svm = precision_score(y_test, y_pred_best_svm)
recall_best_svm = recall_score(y_test, y_pred_best_svm)
f1_best_svm = f1_score(y_test, y_pred_best_svm)

print("Best SVM Model Metrics:")
print("Accuracy:", accuracy_best_svm)
print("Precision:", precision_best_svm)
print("Recall:", recall_best_svm)
print("F1 Score:", f1_best_svm)


Best SVM Model Metrics:
Accuracy: 0.62004662004662
Precision: 0.6188524590163934
Recall: 0.6832579185520362
F1 Score: 0.6494623655913978


In [23]:
# Predict on the test set using the best model
y_pred_best_svm = svm_best.predict(X_train)

# Calculate metrics for the best model
accuracy_best_svm = accuracy_score(y_train, y_pred_best_svm)
precision_best_svm = precision_score(y_train, y_pred_best_svm)
recall_best_svm = recall_score(y_train, y_pred_best_svm)
f1_best_svm = f1_score(y_train, y_pred_best_svm)

print("Best SVM Model Metrics:")
print("Accuracy:", accuracy_best_svm)
print("Precision:", precision_best_svm)
print("Recall:", recall_best_svm)
print("F1 Score:", f1_best_svm)


Best SVM Model Metrics:
Accuracy: 0.7002851957479906
Precision: 0.6844762305427009
Recall: 0.8002951303492376
F1 Score: 0.7378684807256236


In [26]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Using the best parameters to train the SVM model
best_params_svm = {'C': 5, 'gamma': 0.001, 'kernel': 'rbf'}

svm_best_tuned = SVC(**best_params_svm)
svm_best_tuned.fit(X_train, y_train)

# Predict on the test set using the best model
y_pred_svm = svm_best_tuned.predict(X_test)

# Calculate metrics for the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_svm)
precision_best_svm = precision_score(y_test, y_pred_svm)
recall_best_svm = recall_score(y_test, y_pred_svm)
f1_best_svm = f1_score(y_test, y_pred_svm)

print("Best SVM test Metrics:")
print("Accuracy:", accuracy_best_svm)
print("Precision:", precision_best_svm)
print("Recall:", recall_best_svm)
print("F1 Score:", f1_best_svm)

# Predict on the test set using the best model
y_pred_svm = svm_best_tuned.predict(X_train)

# Calculate metrics for the best model
accuracy_best_svm = accuracy_score(y_train, y_pred_svm)
precision_best_svm = precision_score(y_train, y_pred_svm)
recall_best_svm = recall_score(y_train, y_pred_svm)
f1_best_svm = f1_score(y_train, y_pred_svm)

print("\nBest SVM train Metrics:")
print("Accuracy:", accuracy_best_svm)
print("Precision:", precision_best_svm)
print("Recall:", recall_best_svm)
print("F1 Score:", f1_best_svm)


Best SVM test Metrics:
Accuracy: 0.6130536130536131
Precision: 0.6070038910505836
Recall: 0.7058823529411765
F1 Score: 0.6527196652719665

Best SVM train Metrics:
Accuracy: 0.6582836401348198
Precision: 0.6465764657646577
Recall: 0.7757009345794392
F1 Score: 0.7052772808586764


#### Observation: SVM model is overfitting 

# Saving SVM Model

In [31]:
import pickle

# Specify a path
model_filename = 'independant_audio_model.pkl'

# Save the model to the file
with open(model_filename, 'wb') as model_file:
    pickle.dump(svm_best, model_file)
